In [ ]:
import torch
import torch.nn as nn

from torchinfo import summary

In [ ]:
# half a U-net? L-net?
# this network is intended to predict only the radius of a single circle
def build_circle_spotter():
    circle_spotter = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.AdaptiveAvgPool2d((8, 8)),
        nn.Flatten(),
        nn.Linear(in_features=32*(8*8), out_features=1024),
        
        # up to 64 circles
        # train the network to identify up to 8
        # circle radiuses, sorted from largest to smallest
        nn.Linear(in_features=1024, out_features=8),
    )

    return circle_spotter

In [ ]:
summary(build_circle_spotter(), input_size=(2, 1, 128, 128))

In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
def get_empty_image(image_width, image_height):
    # mode="F" for 32-bit floating point pixels
    empty_image = Image.new(
        mode="F",
        size=(image_width, image_height)
    )

    return empty_image

In [ ]:
def draw_a_circle(target_image, circle_e1, circle_e2, circle_radius):
    """
    The most simple image of a circle?
    """
    artist = ImageDraw.ImageDraw(target_image)
    artist.ellipse(
        (
            circle_e1 - circle_radius/2,
            circle_e2 - circle_radius/2,
            circle_e1 + circle_radius/2,
            circle_e2 + circle_radius/2
        ),
        width=1,
        outline=255
    )
    
    return target_image


In [ ]:
def image_of_circles(circle_count):
    """
    The most simple image of a circle?
    """
    image_of_circles = get_empty_image(
        image_width=128,
        image_height=128
    )

    circle_radius_list = list()

    for circle_i in range(circle_count):
        circle_radius = np.random.randint(low=10, high=40)
        circle_radius_list.append(circle_radius)
        draw_a_circle(
            target_image=image_of_circles,
            circle_e1=np.random.randint(low=20, high=80),
            circle_e2=np.random.randint(low=20, high=80),
            circle_radius=circle_radius,
        )

    return (circle_radius_list, image_of_circles)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=2, ncols=2)
for r, c in itertools.product(range(2), range(2)):
    circle_count = np.random.randint(low=1, high=5)
    circle_radiuses, circles_image = image_of_circles(circle_count=circle_count)
    
    print('circle radiuses: {}'.format(circle_radiuses))

    axs[r][c].imshow(circles_image, origin="lower")
    #print(np.array(im))


In [ ]:
# a class to interact with DataLoaders
class CircleImageDataset:
    def __init__(self, circle_image_count):
        self.circle_image_list = list()
        for i in range(circle_image_count):
            
            circle_count = np.random.randint(low=1, high=5)
            circle_radius_list, circles_image = image_of_circles(circle_count=circle_count)

            # sort the circle radiuses in descending order
            # otherwise the training data is a little ambiguous?
            sorted_circle_radius_list = sorted(circle_radius_list, reverse=True)

            # the network output is a 8-element array of circle radiuses
            circle_radiuses = np.zeros((8, ), dtype=np.float32)
            circle_radiuses[:circle_count] = sorted_circle_radius_list
            
            self.circle_image_list.append(
                (
                    # get the right type here - single precision floating point
                    # this depends on how the optimization is handled
                    # but I want to get it right here
                    circle_radiuses,

                    # the PIL image is converted to a 2D numpy array here
                    # in addition an extra dimension is inserted for 'channel'
                    # which PyTorch convolutional networks expect
                    np.expand_dims(
                        np.array(circles_image),
                        axis=0
                    )
                )
            )

    def __getitem__(self, index):
        # self.circle_image_list looks like
        #   [ (radius_0, radius_1, ...), image_0), (radius_0, radius_1, ...), image_1), ...]
        # this dataset returns only (radius, image)
        return self.circle_image_list[index]

    def __len__(self):
        return len(self.circle_image_list)

In [ ]:
def test_circle_image_dataset():
    circle_image_dataset = CircleImageDataset(100)
    print(f"len(circle_image_dataset): {len(circle_image_dataset)}")
    circle_radiuses, circle_image = circle_image_dataset[99]
    print(f"radius      : {circle_radiuses}")
    print(f"image.shape : {circle_image.shape}")

test_circle_image_dataset()

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def test_circle_image_dataloader():
    circle_image_dataloader = DataLoader(CircleImageDataset(circle_image_count=100), batch_size=10)
    for batch in circle_image_dataloader:
        print(f"len(batch): {len(batch)}")
        print(f"len(batch[0]): {len(batch[0])}")
        print(f"batch[0].shape: {batch[0].shape}")
        print(f"len(batch[1]): {len(batch[1])}")
        print(f"batch[1].shape: {batch[1].shape}")

        correct_radii, circle_images = batch

        # note correct_radii.shape does not match predicted_radii.shape
        print(f"correct_radii.shape: {correct_radii.shape}")
        print(f"correct_radii.dtype: {correct_radii.dtype}")
        print(f"circle_images.shape: {circle_images.shape}")

        test_circle_spotter = build_circle_spotter()
        predicted_radii = test_circle_spotter(circle_images)
        print(f"predicted_radii.shape: {predicted_radii.shape}")
        print(f"predicted_radii.dtype: {predicted_radii.dtype}")
        
        break

test_circle_image_dataloader()

In [ ]:
train_circle_image_loader = DataLoader(CircleImageDataset(circle_image_count=10000), batch_size=100)
test_circle_image_loader = DataLoader(CircleImageDataset(circle_image_count=1000), batch_size=100)
#validate_circle_image_loader = DataLoader(CircleImageDataset(circle_image_count=1000), batch_size=100)

In [ ]:
len(train_circle_image_loader.dataset)

In [ ]:
def train(
    circle_spotter_model,
    optimizer,
    loss_function,
    train_dataloader,
    test_dataloader,
    epoch_count
):
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    circle_spotter_model.to(device)

    for epoch_i in range(epoch_count):
        training_loss = 0.0
        circle_spotter_model.train()
        for correct_radii, circle_images in train_dataloader:
            optimizer.zero_grad()

            # torch calls circle_images 'inputs'
            circle_images = circle_images.to(device)
            # make the correct_radii array match predicted_radii.shape
            #correct_radii = torch.unsqueeze(correct_radii, 1)
            correct_radii = correct_radii.to(device)

            predicted_radii = circle_spotter_model(circle_images)
            
            loss = loss_function(predicted_radii, correct_radii)
            loss.backward()
            optimizer.step()

            training_loss += loss.data.item()

        training_loss /= len(train_circle_image_loader.dataset)

        circle_spotter_model.eval()
        test_loss = 0.0
        num_correct = 0.0
        num_examples = 0.0
        for correct_radii, circle_images in test_dataloader:

            # torch calls circle_images 'inputs'
            circle_images = circle_images.to(device)
            #inputs = inputs.to(device)
            # make correct_radii have the same shape as predicted_radii
            #correct_radii = torch.unsqueeze(correct_radii, 1)
            correct_radii = correct_radii.to(device)

            predicted_radii = circle_spotter_model(circle_images)

            loss = loss_function(predicted_radii, correct_radii)
            test_loss += loss.data.item()
            # call a predicted radius "correct" if it is within 10% of the correct radius
            # for example
            #   correct radius:   [ 33.,     20.,     18.,     13.,     0.,       0.,      0.,        0.]
            #   predicted radius: [ 2.8e+01, 2.4e+01, 1.9e+01, 1.1e+01, 2.1e-02, -4.3e-03, -1.4e-02,  1.6e-02]
            #   percent wrong: |33 - 28|/33 = 5/30 = 1/6 = 0.167 = 16.7%
            # meaning the predicted radius 28 is 16.7% wrong
            percent_wrong = (torch.abs(correct_radii - predicted_radii) / correct_radii)
            percent_wrong = percent_wrong.cpu()
            num_correct += np.count_nonzero((percent_wrong <= 0.1).numpy())
            #correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)
            #num_correct += torch.sum(correct).item()
            num_examples += circle_images.shape[0]

        test_loss /= len(test_dataloader.dataset)

        print(
            #'Epoch: {}, Training Loss: {:.2f}, Test Loss: {:.2f}, percent_wrong = {}'.format(
            'Epoch: {}, Training Loss: {:.2f}, Test Loss: {:.2f}'.format(
                epoch_i, training_loss, test_loss
            )
        )

In [ ]:
import torch.optim

circle_spotter = build_circle_spotter()
train(
    circle_spotter,
    torch.optim.Adam(circle_spotter.parameters()),
    torch.nn.MSELoss(),
    train_circle_image_loader,
    test_circle_image_loader,
    epoch_count=100
)

In [ ]:
# try out the circle spotter
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

    circle_spotter.eval()
a_circle_image_dataloader = DataLoader(CircleImageDataset(10), batch_size=1)
for a_circle_radius, a_circle_image in a_circle_image_dataloader:
    a_circle_image = a_circle_image.to(device)

    print(a_circle_image.shape)
    predicted_radius = circle_spotter(a_circle_image)
    
    print(f"correct radius   : {a_circle_radius}")
    print(f"predicted radius : {predicted_radius}")

In [ ]:
print(circle_spotter)